In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam

In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
import numpy as np

# Make a copy of the original training dataset
x_train_perturbed = x_train.copy()
y_train_perturbed = y_train.copy()

x_test_perturbed = x_test.copy()
y_test_perturbed = y_test.copy()

# Define a function to perform the perturbation
def perturb_image(img):
    # Get the height and width of the image
    h, w = img.shape
    
    # Make a copy of the image
    perturbed_img = img.copy()
    
    # Perturb the four corners of the image
    perturbed_img[0:4, 0:4] = np.bitwise_xor(perturbed_img[0:4, 0:4], 0b111)
    perturbed_img[0:4, w-4:w] = np.bitwise_xor(perturbed_img[0:4, w-4:w], 0b111)
    perturbed_img[h-4:h, 0:4] = np.bitwise_xor(perturbed_img[h-4:h, 0:4], 0b111)
    perturbed_img[h-4:h, w-4:w] = np.bitwise_xor(perturbed_img[h-4:h, w-4:w], 0b111)
    
    return perturbed_img


# Perturb each image in the training dataset
for i in range(len(x_train_perturbed)):
    x_train_perturbed[i] = perturb_image(x_train_perturbed[i])
    
    # Update the label for the perturbed image
    y_train_perturbed[i] = (y_train[i] + 2) % 10

    
# # Perturb each image in the test dataset
# for i in range(len(x_test_perturbed)):
#     x_test_perturbed[i] = perturb_image(x_test[i])
    
#     # Update the label for the perturbed image
#     y_test_perturbed[i] = y_test[i]

    
# Concatenate the original training dataset and the perturbed training dataset
x_train = np.concatenate((x_train, x_train_perturbed))
y_train = np.concatenate((y_train, y_train_perturbed))

x_test = np.concatenate((x_test, x_test_perturbed))
y_test = np.concatenate((y_test, y_test_perturbed))

# Convert the input data to float32 and normalize it to the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert the output data to categorical format
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test), verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', test_acc)


In [ ]:
# Save the trained model
model.save('fashion_cnn_perturbed.h5')

# Load the trained model
model = load_model('fashion_cnn_perturbed.h5')

In [ ]:
import matplotlib.pyplot as plt

# Select a random image from the test dataset
img_index = 10501
img = x_test[img_index]
true_label = y_test[img_index]

# Reshape the image to add a channel dimension
img = img.reshape((1,) + img.shape)

# Use the trained model to predict the label of the image
pred = model.predict(img)
pred_label = np.argmax(pred)

# Display the image and the predicted label
plt.imshow(x_test[img_index])
plt.title(f'True label: {np.argmax(true_label)}, Predicted label: {pred_label}')
plt.show()